In [15]:
import pretty_midi
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import librosa.display

ModuleNotFoundError: No module named 'librosa'

In [9]:
folder_midi = Path().cwd()
print(folder_midi)

/home/tim/Documents/midi-explore/notebooks/scratch


In [12]:
pm = pretty_midi.PrettyMIDI(str(folder_midi / '367sonat_a1.mid'))

In [13]:
plt.figure(figsize=(12, 4))
plot_piano_roll(pm, 24, 84)